In [1]:
import src.requestcompletion as rc
from typing import List, Any, Dict
from examples.sample_agents import NotionAgent
from src.requestcompletion.llm import MessageHistory, UserMessage
from src.requestcompletion.nodes.library import from_function
from examples.sample_tools.notion_tools import find_page, create_page, add_block, find_user, tag_user
from src.requestcompletion.visuals.agent_viewer import AgentViewer

## Functions to be used as tools for travel planner

In [2]:
def available_locations() -> List[str]:
    """Returns a list of available locations.
    Args:
    Returns:
        List[str]: A list of available locations.
    """
    return [
        "New York",
        "Los Angeles",
        "Chicago",
        "Delhi",
        "Mumbai",
        "Bangalore",
        "Paris",
        "Denmark",
        "Sweden",
        "Norway",
        "Germany",
    ]

def currency_used(location: str) -> str:
    """Returns the currency used in a location.
    Args:
        location (str): The location to get the currency used for.
    Returns:
        str: The currency used in the location.
    """
    currency_map = {
        "New York": "USD",
        "Los Angeles": "USD",
        "Chicago": "USD",
        "Delhi": "INR",
        "Mumbai": "INR",
        "Bangalore": "INR",
        "Paris": "EUR",
        "Denmark": "EUR",
        "Sweden": "EUR",
        "Norway": "EUR",
        "Germany": "EUR",
    }
    used_currency = currency_map.get(location)
    if used_currency is None:
        raise ValueError(f"Currency not available for location: {location}")
    return used_currency

def average_location_cost(location: str, num_days: int) -> float:
    """Returns the average cost of living in a location for a given number of days.
    Args:
        location (str): The location to get the cost of living for.
        num_days (int): The number of days for the trip.
    Returns:
        float: The average cost of living in the location.
    """
    daily_costs = {
        "New York": 200.0,
        "Los Angeles": 180.0,
        "Chicago": 150.0,
        "Delhi": 50.0,
        "Mumbai": 55.0,
        "Bangalore": 60.0,
        "Paris": 220.0,
        "Denmark": 250.0,
        "Sweden": 240.0,
        "Norway": 230.0,
        "Germany": 210.0,
    }
    daily_cost = daily_costs.get(location)
    if daily_cost is None:
        raise ValueError(f"Cost information not available for location: {location}")
    return daily_cost * num_days

def convert_currency(amount: float, from_currency: str, to_currency: str) -> float:
    """Converts currency using a static exchange rate (for testing purposes).
    Args:
        amount (float): The amount to convert.
        from_currency (str): The currency to convert from.
        to_currency (str): The currency to convert to.
    Returns:
        float: The converted amount.
    Raises:
        ValueError: If the exchange rate is not available.
    """
    exchange_rates = {
        ("USD", "EUR"): 0.85,
        ("EUR", "USD"): 1.1765,
        ("USD", "INR"): 83.0,
        ("INR", "USD"): 0.01205,
        ("EUR", "INR"): 98.0,
        ("INR", "EUR"): 0.0102,
    }

    rate = exchange_rates.get((from_currency, to_currency))
    if rate is None:
        raise ValueError("Exchange rate not available")
    return amount * rate



In [3]:
convert_currency_node = rc.library.from_function(convert_currency)
available_locations_node = rc.library.from_function(available_locations)
currency_used_node = rc.library.from_function(currency_used)
average_location_cost_node = rc.library.from_function(average_location_cost)    

## Travel Planner Agent

In [ ]:
travel_planner_agent = rc.library.tool_call_llm(connected_nodes={convert_currency_node, available_locations_node, currency_used_node, average_location_cost_node}, 
                                               pretty_name="Travel Planner Node", 
                                               system_message=rc.llm.SystemMessage("You are a travel planner that will plan a trip. you have access to AvailableLocations, ConvertCurrency, CurrencyUsed and AverageLocationCost tools. Use them when you need to."),
                                               model=rc.llm.OpenAILLM("gpt-4o"))

In [ ]:
USER_PROMPT = "I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR. " \
"Please make a notion page called Amir Travel Summary under the page 'Agent Demo Root' and write my travel summary in it."

In [ ]:
with rc.Runner() as runner:
    result = await runner.run(travel_planner_agent, message_history=rc.llm.MessageHistory([rc.llm.UserMessage(USER_PROMPT)]))

## Wrapping Notion agent with Travel Planner agent inside a top level node

In [ ]:
USER_PROMPT = "I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR. " \
"Please make a notion page called Amir Travel Summary under the page 'Agent Demo Root' and write my travel summary in it."

In [ ]:
async def top_level_node():
    travel_plan = await rc.call(travel_planner_agent, 
                           message_history=MessageHistory([UserMessage(USER_PROMPT)]))

    
    await rc.call(NotionAgent, message_history=MessageHistory([UserMessage(USER_PROMPT),
                                                               UserMessage(f"Travel Plan: {travel_plan}")]))


In [ ]:
TravelSummarizerNotion = from_function(top_level_node)

with rc.Runner(rc.ExecutorConfig(timeout=50)) as runner:
    result = await runner.run(TravelSummarizerNotion)

## Trying to make an agentic tool out of NotionAgent

In [4]:
class NotionAgenticTool(rc.library.ToolCallLLM):
    def __init__(self, message_history: MessageHistory):
        super().__init__(message_history=message_history, model=rc.llm.OpenAILLM("gpt-4o"))

    def connected_nodes(self):
        return {
            from_function(find_page),
            from_function(create_page),
            from_function(add_block),
            from_function(find_user),
            from_function(tag_user),
            }
    
    @classmethod
    def tool_info(cls) -> rc.llm.Tool:
        return rc.llm.Tool(
            name="NotionAgenticTool",
            detail="A notion tool that can find pages, create pages, add blocks, find users, and tag users.",
            parameters={rc.llm.Parameter(name="notion_task_request", param_type="string", description="The detailed task prompt for the task to be completed.")},
        )

    @classmethod
    def prepare_tool(cls, tool_parameters: Dict[str, Any]):
        message_hist = MessageHistory(
            [UserMessage(f"Request Prompt: '{tool_parameters['notion_task_request']}'")]
        )
        return cls(message_hist)
    
    @classmethod
    def pretty_name(cls) -> str:
        return "Notion Agentic Tool"


In [5]:
# USER_PROMPT = "Give me a sample travel summary, very short. Please make a notion page called Aryan Travel Summary under the page 'Agent Demo Root' and write my travel summary in it." \
USER_PROMPT = "I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR. Please provide me with a budget summary for the trip in INR. Please make a notion page called Aryan Travel Summary under the page 'Agent Demo Root' and write my travel summary in it."

In [6]:
TravelAgent = rc.library.tool_call_llm(connected_nodes={convert_currency_node, available_locations_node, currency_used_node, average_location_cost_node, NotionAgenticTool}, 
                                        pretty_name="Travel Agent", 
                                        system_message=rc.llm.SystemMessage("You are a travel planner that will plan a trip. you have access to AvailableLocations, ConvertCurrency, CurrencyUsed, AverageLocationCost and NotionAgenticTool tools. Use them when you need to."),
                                        model=rc.llm.OpenAILLM("gpt-4o"))

In [7]:
with rc.Runner() as runner:
    result = await runner.run(TravelAgent, message_history=rc.llm.MessageHistory([rc.llm.UserMessage(USER_PROMPT)]))

[+13.247 s] RC.RUNNER   : INFO     - START CREATED Travel Agent - (, message_history=user: I live in Delhi. I am going to travel to Denmark for 3 days, followed by Germany for 2 days and finally New York for 4 days. Please provide me with a budget summary for the trip in INR. Please provide me with a budget summary for the trip in INR. Please make a notion page called Aryan Travel Summary under the page 'Agent Demo Root' and write my travel summary in it.)
[+14.233 s] RC.RUNNER   : INFO     - Travel Agent CREATED available_locations Node - ({}, )
[+14.235 s] RC.RUNNER   : INFO     - available_locations Node DONE ['New York', 'Los Angeles', 'Chicago', 'Delhi', 'Mumbai', 'Bangalore', 'Paris', 'Denmark', 'Sweden', 'Norway', 'Germany']
[+16.854 s] RC.RUNNER   : INFO     - Travel Agent CREATED average_location_cost Node - ({'num_days': 3, 'location': 'Denmark'}, )
[+16.857 s] RC.RUNNER   : INFO     - average_location_cost Node DONE 750.0
[+18.721 s] RC.RUNNER   : INFO     - Travel Agent CRE

In [9]:
viewer = AgentViewer(
        stamps=result.all_stamps,
        request_heap=result.request_heap,
        node_heap=result.node_heap
    )
viewer.display_graph()